# Data Access and Visualisation

In [ ]:
from urllib.parse import urljoin, quote

from owslib.csw import CatalogueServiceWeb
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
import lxml.etree
import requests
from tifffile import imread
from io import BytesIO, StringIO
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# some base settings
domain = "demo.eoepca.org"
data_access_base = f"data-access.{domain}"
ows_endpoint = f"https://{data_access_base}/ows"

In [ ]:
# connect to the OWS endpoint using WMS
wms = WebMapService(ows_endpoint, version='1.3.0')

In [ ]:
# list all advertised layers
list(wms.contents)

In [ ]:
# explore the spatio-temporal bounds of the S2L1C layer
layer = "S2L1C"
bbox = wms.contents[layer].boundingBoxWGS84
time = wms.contents[layer].timepositions
bbox, time

In [ ]:
# helper function to do WMS GetMap requests and directly display the result in the notebook
def get_map(wms, layers, bbox, styles=None, size=None, srs='EPSG:4326', transparent=True, format="image/png", **kwargs):
    # if no specific size is passed, calculate one fitting the aspect ratio of the bbox
    if size is None:
        ratio = (bbox[3] - bbox[1])/(bbox[2] - bbox[0])
        width = 600
        height = int(ratio * width)
        size = (width, height)

    # qol helpers
    if isinstance(layers, str):
        layers = [layers]
    if isinstance(styles, str):
        styles = [styles]

    result = wms.getmap(
        layers=layers,
        styles=styles,
        size=size,
        srs=srs,
        bbox=bbox,
        format=format,
        transparent=transparent,
        **kwargs
    )
    return Image(result.read())

# Browsing

With the help of WMS the contents of the data access can be visually browsed. Additional filters like `time` and `cql` can be applied to drill down to the scenes of interest

In [ ]:
# get an overview of all registered scenes
get_map(wms, "S2L1C__outlines", bbox)

In [ ]:
# filter scenes by time
get_map(wms, "S2L1C__outlines", bbox, time="2020-09-01T00:00:00Z/2020-10-01T00:00:00Z")

In [ ]:
# filter scenes by using time + cql to get scenes with cloud cover less than 10%
get_map(wms, "S2L1C__outlines", bbox, time="2020-09-01T00:00:00Z/2020-10-01T00:00:00Z", cql="product_metadata__cloud_cover < 10")

# Visualization

The EOEPCA data access allows for predefined as well as on-demand visualizations, on both a whole collections or single scenes. In order to not overwhelm the service with too many requests, we will limit the requests to a single scene.

Scene layers are not advertised, but can be discovered when using a CQL query in the `GetCapabilities` request.

Here we will explore the different visualization methods:

In [ ]:
scene_id = "S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255.SAFE"
cql = quote(f'identifier = "{scene_id}"')

scene_wms = WebMapService(f'{ows_endpoint}?cql={cql}')

list(scene_wms.contents)

In [ ]:
scene_bbox = scene_wms.contents[scene_id].boundingBoxWGS84

In [ ]:
# default rendering using TCI file
get_map(scene_wms, scene_id, scene_bbox)

In [ ]:
# True color composite, using RGB bands B04, B03, B02 with adjusted color stretches
get_map(scene_wms, f"{scene_id}__TRUE_COLOR", scene_bbox)

In [ ]:
# False color composite using NirRG bands B08, B04, B03
get_map(scene_wms, f"{scene_id}__FALSE_COLOR", scene_bbox)

In [ ]:
# NDVI layer: (B08-B04)/(B08+B04)
get_map(scene_wms, f"{scene_id}__NDVI", scene_bbox)

In [ ]:
# NDVI layer with viridis color scale
get_map(scene_wms, f"{scene_id}__NDVI", scene_bbox, styles=["viridis"])

In [ ]:
# Single band access with colorscale applied and custom value stretch
get_map(scene_wms, scene_id, scene_bbox, styles=["plasma"], dim_bands="B08", dim_range="0 4000")

In [ ]:
# SWIR rendering (B12, B08, B04)
get_map(scene_wms, scene_id, scene_bbox, dim_bands="B12,B08,B04", dim_range="0 4000,0 4000,0 4000")

In [ ]:
import folium
import folium.plugins.timestamped_wmstilelayer


centre_lat = scene_bbox[1] + (scene_bbox[3] - scene_bbox[1])/2
centre_long = scene_bbox[0] + (scene_bbox[2] - scene_bbox[0])/2
m = folium.Map(location=[centre_lat, centre_long], zoom_start=7, tiles=None)

folium.raster_layers.WmsTileLayer(
    url="https://a.tiles.maps.eox.at",
    layers='terrain-light_3857',
    name='terrain-light',
    fmt='image/jpeg',
).add_to(m)
folium.raster_layers.WmsTileLayer(
    url=ows_endpoint,
    layers=f"{scene_id}__TRUE_COLOR",
    name=f"{scene_id}__TRUE_COLOR",
    fmt='image/png',
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)
m

# Data Access

In order to access the actual data contained in the band files, we can use the WCS protocol. This allows us to access any data file via a common interface and do dynamic subsets.

To list the available coverages of a scene, we can send a `DescribeEOCoverageSet` request.

In [ ]:
response = requests.get(ows_endpoint, params={"service": "WCS", "version": "2.0.0", "request": "DescribeEOCoverageSet", "eoid": scene_id})
tree = lxml.etree.fromstring(response.content)
tree.xpath("wcs:CoverageDescriptions/wcs:CoverageDescription/@gml:id", namespaces=tree.nsmap)

Now that we have the Coverage IDs we can send `GetCoverage` requests to download the data, to do our own calculations with it:

In [ ]:
# convenience function to request a coverage and read the TIFF
def get_coverage(ows_endpoint, params):
    response = requests.get(ows_endpoint, params=params)
    return imread(BytesIO(response.content))

In [ ]:
b04 = get_coverage(ows_endpoint, params={"service": "WCS", "version": "2.0.0", "request": "GetCoverage", "coverageid": "S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255.SAFE_B04", "format": "image/tiff", "scaleFactor": "0.10"})
b08 = get_coverage(ows_endpoint, params={"service": "WCS", "version": "2.0.0", "request": "GetCoverage", "coverageid": "S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255.SAFE_B08", "format": "image/tiff", "scaleFactor": "0.10"})

In [ ]:
b04 = b04.astype("float64")
b08 = b08.astype("float64")

ndvi = (b08 - b04) / (b08 + b04)
plt.imshow(ndvi, aspect="auto")